<a href="https://colab.research.google.com/github/jermwatt/morphi_lab/blob/object_diffusion_collab_demo/object_diffusion_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Single object diffusion demo

This notebook allows for experimentation with object detection + segmentation --> diffusion.

To use it first:

- select a gpu runtime: make sure you have chosen a GPU runtime from the options above - choose A100 if available.
- activate the first few cells to 
  - perform a machine check: make sure your runtime is employing a GPU with sufficient memory
  - download associated files and installs
- start experimenting!

### Some example experiments

Here are the sort of experiments you can perform in this notebook.  On the left below we have a picture with two default yolo-recognizable objects: a `person` and a `donut` (there are around 70 pre-trained objects you can play with by default).  Using the functionality here you can replace either one independently.

So you can replace the `person` as shown below.

<img src='https://github.com/jermwatt/morphi_lab/blob/object_diffusion_collab_demo/test_data/example_person_replacement.png?raw=true'>

Or just as easily the `donut` as shown here.

<img src='https://github.com/jermwatt/morphi_lab/blob/ebc31b93c4e9ec941483723ab2cb99c38ce4929a/test_data/example_donut_replacement.png?raw=true'>


## 1.  machine setup and installs

### 1.1. pre-launch machine check

Activate the code block below to double check that you are using a GPU runtime for the experiments.  You don't have to use one if you can't, but the experiments will run significantly faster (in particular the diffusion steps) if you are using one.

In [ ]:
# check for GPU runtime
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# check for memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

### 1.2. installs

Activate the cell below to install all necessary packages to your collab enviroment for these experiments.

In [ ]:
# install reqiured libraries 
!pip install "ultralytics==8.0.111" "transformers==4.29.2" "timm==0.9.2" "diffusers==0.16.1" "safetensors==0.3.1" "accelerate==0.19.0"

Activate the cell below to pull all helper functions required to run this demo.

In [ ]:
# pull some images from the web
import urllib.request

def download_file(url, output_path):
    urllib.request.urlretrieve(url, output_path)

# pull segmentation module
url = "https://raw.githubusercontent.com/jermwatt/morphi_lab/object_diffusion_collab_demo/segmenter.py"
local_path = "/content/segmenter.py"
download_file(url,local_path)

# pull diffusion module
url = "https://raw.githubusercontent.com/jermwatt/morphi_lab/object_diffusion_collab_demo/diffuser.py"
local_path = "/content/diffuser.py"
download_file(url,local_path)

# pull utilities module
url = "https://raw.githubusercontent.com/jermwatt/morphi_lab/object_diffusion_collab_demo/utilities.py"
local_path = "/content/utilities.py"
download_file(url,local_path)

# pull main module
url = "https://raw.githubusercontent.com/jermwatt/morphi_lab/object_diffusion_collab_demo/main.py"
local_path = "/content/main.py"
download_file(url,local_path)

Activate the cell below to pull in a a sample images to experiment with.

In [ ]:
# man holding donut - we'll use this one for testing
url = "https://www.shutterstock.com/image-photo/surprised-young-man-holding-donut-260nw-586330142.jpg"
output_path = "/content/test_donut.png"
download_file(url, output_path)

### 1.3.  Module setup

Activate the next cell to load in the required functionality from your newly installed modules.

The first time you do this both a Yolo and Diffusion models (so will take 30 seconds to a minute or two to execute).

In [ ]:
# import segmentation and diffusion modules via main module
from main import main

## 2. start experimenting!

Below we print out the set of available labels you can choose from in your test images.

In [ ]:
from segmenter import label_lookup_dict
print('available labels to choose from for test image segmentation and diffusion')
print(list(label_lookup_dict.keys()))

### 2.1. diffuse a segmented object

Let's segment out and diffuse the `person` in our test image.

In [ ]:
## segment the donut out of the test image
# define parameters
img_path = "/content/test_donut.png"
label = 'person'
prompt='an ape, smiling, high resolution, holding something'
seed=3433

# segment and diffuse
main(img_path=img_path,
     label=label,
     prompt=prompt,
     seed=seed)

Notice the fixed `seed` I am inputing (to keep the output look the same).  If you want a random seed - simply omit this option.


Let's try this below.

In [ ]:
## segment the donut out of the test image
# define parameters
img_path = "/content/test_donut.png"
label = 'person'
prompt='an ape, smiling, high resolution, holding something'

# segment and diffuse
main(img_path=img_path,
     label=label,
     prompt=prompt)

Let's try the same image - different object.

Now let's replace the `donut`. 

In [ ]:
## segment the donut out of the test image
# define parameters
img_path = "/content/test_donut.png"
label = 'donut'
prompt='an orange, fruit, high resolution'
seed=3433

# segment and diffuse
main(img_path=img_path,
     label=label,
     prompt=prompt,
     seed=seed)

Note there are several optinal arguments you can play with here visa-vis the diffuser that I didn't mess with in this example, including
  - `negative_prompt` (default is None)
  - `num_inference_steps` (default is 100)

### 2.4. Trying your own

You can try your own image by pulling one via the `download_file` function above, or by downloading and dragging / dropping into the `/content` directory by hand.  Note for now the diffuser only accepts `.png` and `.jpg/jpeg`.

You then use the same function calls executed above - tailored to your specific image.

I'm going to use the url downloader to get a picture of a cat stealing pizza.

In [ ]:
# pizza stealing cat
url = "https://i.ytimg.com/vi/IflVaMkULxg/hqdefault.jpg"
output_path = "/content/pizza_cat.jpg"
download_file(url, output_path)

Let's first replace the cat!

In [ ]:
## segment and diffuse cat
# define parameters
img_path = "/content/pizza_cat.jpg"
label = 'cat'
prompt='anime nerd, high resolution'
seed=3433

# segment and diffuse
main(img_path=img_path,
     label=label,
     prompt=prompt,
     seed=seed)

Now let's try replacing the pizza.

In [ ]:
## segment and diffuse pizza
# define parameters
img_path = "/content/pizza_cat.jpg"
label = 'pizza'
prompt='a pile of cash, fanned out, high resolution'
negative_prompt='low quality'
num_inference_steps=100
seed=4637

# segment and diffuse
main(img_path=img_path,
     label=label,
     prompt=prompt,
     seed=seed,
     negative_prompt=negative_prompt,
     num_inference_steps=num_inference_steps)